Conversion + Segmentation of LIF files
====
- Read in the LIF files
- Save them as TIFs (they're nicer for me to work with)
- Segment them with my pipeline

In [ ]:
"""
Get paths to LIF files
"""

import pathlib

parent_dir = pathlib.Path(
    "~/zebrafish_rdsf/Carran/Postgrad/Scale images from WT_spp1_sost"
).expanduser()
assert parent_dir.exists()

lif_paths = list(parent_dir.glob("*"))

In [ ]:
"""
Get rid of some weird ones
"""
lif_paths = [l for l in lif_paths if "out of focus" not in str(l) and not l.is_dir() and not l.stem == ".DS_Store"]
lif_paths

In [ ]:
"""
For now, select only the ALP stained ones, because they're easier to segment
"""

import pandas as pd


def get_stain(name: str) -> str:
    """
    Expect either KOSSA, ALP or TRAP
    """
    # Expect name to always end in .lif
    (stem, ending) = name.split(".lif")
    assert not ending, f"{stem}, {ending}"

    vk, alp, trap = False, False, False
    if "KOSSA" in stem or "VK" in stem:
        vk = True
    if "ALP" in stem:
        alp = True
    if "TRAP" in stem:
        trap = True
    
    if vk + alp + trap != 1:
        raise ValueError(f"Unknown stain for {stem}; {vk=}, {alp=}, {trap=}")
    
    if vk:
        return "KOSSA"
    if alp:
        return "ALP"
    return "TRAP"
    


lif_df = pd.DataFrame(columns=["path"])
lif_df["path"] = lif_paths
lif_df["name"] = lif_df["path"].apply(lambda p: pathlib.Path(p).name)
lif_df["stain"] = lif_df["name"].apply(get_stain)

lif_df

In [ ]:
"""
Extract ages
"""


def get_year(stem):
    """
    Extract years from a filepath
    """
    if "year" not in stem:
        return 0

    year_str, _ = stem.split("year")
    return int(year_str.strip()[-1])


def get_month(stem):
    """
    Extract months from a filepath
    """
    if not "month" in stem:
        return 0
    month_str, _ = stem.split("month")
    month_str = month_str.strip()

    # We might have a 2-digit month, so check for that here
    if month_str[-2].isdigit():
        return int(month_str[-2:])
    return int(month_str[-1])


def get_age(stem):
    """Get age in months"""
    return 12 * get_year(stem) + get_month(stem)


lif_df["age"] = lif_df["name"].apply(get_age)

In [ ]:
"""
Select only the ALP ones, for now
"""
alp_df = lif_df[lif_df["stain"] == "ALP"]
alp_df

In [ ]:
"""
Read them into arrays
"""

from tqdm import tqdm
from scale_morphology.scales import read

names, imgs, lifs = [], [], []
for path in tqdm(alp_df["path"]):
    name, img = zip(*read.read_lif(path))

    names += name
    imgs += img
    lifs += [path.name for _ in name]

In [ ]:
"""
Munge names and save as TIFs
"""

import tifffile

out_dir = parent_dir / "TIFs"
try:
    out_dir.mkdir(exist_ok=False)
    for name, img, lif in zip(tqdm(names), imgs, lifs, strict=True):
        path = (
            name
            + "__"
            + lif.replace(".lif", "").replace(".", "_").replace(" ", "_")
            + ".tif"
        )
        tifffile.imwrite(out_dir / path, img)
except FileExistsError:
    print("dir", out_dir, "exists")